In [ ]:
!kaggle datasets download -d ismailnasri20/driver-drowsiness-dataset-ddd

Dataset URL: https://www.kaggle.com/datasets/ismailnasri20/driver-drowsiness-dataset-ddd
License(s): unknown
100% 2.57G/2.58G [00:12<00:00, 232MB/s]
100% 2.58G/2.58G [00:12<00:00, 214MB/s]


In [ ]:
import os
import zipfile

# Path to the downloaded file
zip_path = 'driver-drowsiness-dataset-ddd.zip'  # Corrected the missing closing quote
extract_path = 'DrowsyImages'

# Create the directory for extraction if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the paths to the dataset and the target directories
dataset_path = 'DrowsyImages/Driver Drowsiness Dataset (DDD)'
output_dir = 'DrowsyTrainer'
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'validation')
test_dir = os.path.join(output_dir, 'test')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create subdirectories for each class
classes = ['Drowsy', 'Non Drowsy']
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Function to split dataset
def split_dataset(class_name):
    class_path = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
    train_imgs, temp_imgs = train_test_split(images, test_size=0.4, random_state=42)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

    # Move images to train, validation, and test directories
    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, class_name, os.path.basename(img)))
    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, class_name, os.path.basename(img)))
    for img in test_imgs:
        shutil.copy(img, os.path.join(test_dir, class_name, os.path.basename(img)))

# Split each class
for cls in classes:
    split_dataset(cls)

print('Dataset organized into train, validation, and test directories.')

Dataset organized into train, validation, and test directories.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Add, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.layers import LeakyReLU

In [ ]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
model = Sequential([

    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 62, 62, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 29, 29, 64)        256       
 chNormalization)                                                
                                                        

In [ ]:
train_data = data_gen.flow_from_directory(
    'DrowsyTrainer/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 20061 images belonging to 2 classes.


In [ ]:
val_data = data_gen.flow_from_directory(
    'DrowsyTrainer/validation',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1671 images belonging to 2 classes.


In [ ]:
test_data = data_gen.flow_from_directory(
    'DrowsyTrainer/test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
)

Found 8359 images belonging to 2 classes.


In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=25, batch_size=32)

Epoch 1/25
627/627 [==============================] - 87s 138ms/step - loss: 0.1805 - accuracy: 0.9308 - val_loss: 1.8189 - val_accuracy: 0.5075
Epoch 2/25
627/627 [==============================] - 86s 138ms/step - loss: 0.0618 - accuracy: 0.9797 - val_loss: 0.8889 - val_accuracy: 0.7385
Epoch 3/25
627/627 [==============================] - 86s 137ms/step - loss: 0.0444 - accuracy: 0.9865 - val_loss: 3.1559 - val_accuracy: 0.5206
Epoch 4/25
627/627 [==============================] - 87s 138ms/step - loss: 0.0269 - accuracy: 0.9917 - val_loss: 4.1373 - val_accuracy: 0.5332
Epoch 5/25
627/627 [==============================] - 86s 138ms/step - loss: 0.0293 - accuracy: 0.9918 - val_loss: 2.2315 - val_accuracy: 0.7026
Epoch 6/25
627/627 [==============================] - 87s 138ms/step - loss: 0.0244 - accuracy: 0.9919 - val_loss: 3.2843 - val_accuracy: 0.6403
Epoch 7/25
627/627 [==============================] - 85s 136ms/step - loss: 0.0376 - accuracy: 0.9891 - val_loss: 7.0607 - val_ac

In [ ]:
test_loss, test_acc = model.evaluate(test_data, steps=test_data.samples // test_data.batch_size)
print(f"Test Accuracy: {test_acc}")

261/261 [==============================] - 30s 117ms/step - loss: 1.2752 - accuracy: 0.9113
Test Accuracy: 0.9112787246704102


In [ ]:
model.save('drowsiness_detection_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save_weights('drowsiness_detection_weights.h5')

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred_prob = model.predict(test_data, steps=test_data.samples // test_data.batch_size)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)
y_true = test_data.classes

report = classification_report(y_true[:len(y_pred)], y_pred, target_names=['Non Drowsy', 'Drowsy'])

print(report)

261/261 [==============================] - 31s 117ms/step
              precision    recall  f1-score   support

  Non Drowsy       0.53      0.47      0.50      4470
      Drowsy       0.46      0.53      0.49      3882

    accuracy                           0.50      8352
   macro avg       0.50      0.50      0.50      8352
weighted avg       0.50      0.50      0.50      8352

